# AutoGen Basic Sample (SiliconFlow)

在这个代码示例中，您将使用[AutoGen](https://aka.ms/ai-agents/autogen)人工智能框架来创建一个基本代理。

本示例的目标是向您展示我们将在实现不同代理模式时使用的步骤。

## 导入必要的 Python Packages

In [2]:
import os
from dotenv import load_dotenv

from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_core import CancellationToken

from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console

from IPython.display import display, HTML

## 创建 Client (SiliconFlow)

请在本地 `.env` 中设置 `SILICONFLOW_API_KEY`，并使用 `base_url="https://api.siliconflow.cn/v1"`。可根据需要替换模型（示例使用 `Qwen/Qwen3-VL-8B-Instruct`）。

In [7]:
load_dotenv()
client = OpenAIChatCompletionClient(
    model="Qwen/Qwen3-VL-8B-Instruct",
    api_key=os.getenv("SILICONFLOW_API_KEY"),
    base_url="https://api.siliconflow.cn/v1",
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

# Quick test
result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

finish_reason='stop' content='The capital of France is **Paris**.\n\nParis is not only the political and administrative center of France but also its cultural, economic, and historical heart. Known for landmarks like the Eiffel Tower, Louvre Museum, Notre-Dame Cathedral, and the Champs-Élysées, Paris is one of the most visited cities in the world and a global symbol of art, fashion, and cuisine.\n\n✅ *Answer: Paris*' usage=RequestUsage(prompt_tokens=15, completion_tokens=88) cached=False logprobs=None thought=None


## 定义 Agent

现在我们已经设置了 `client` 并确认其正常运行，接下来我们创建一个 `AssistantAgent`。每个代理都可以被分配一个：
- **name** - 一个简短的名称，便于在多代理流程中引用。
- **model_client** - 你在前面步骤中创建的客户端。
- **tools** - 代理可以使用的工具，用于完成任务。
- **system_message** - 定义大语言模型任务、行为和语气的元提示。

你可以更改系统消息以查看大语言模型的响应。我们将在第4课中讲解 `tools`。

In [4]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[],
    system_message="You are a travel agent that plans great vacations",
)

## 运行 Agent

下面的函数将运行代理。我们使用 `on_message` 方法将新的消息更新到代理的状态中。

在此情况下，我们将状态更新为用户的新消息 `"Plan me a great sunny vacation"`。

你可以更改消息内容，以查看LLM如何做出不同的回应。

In [9]:
async def assistant_run():
    # Define the query
    user_query = "给我计划一个阳光明媚的假期，包括景点和活动。全国各地都行，下周就出发"

    # Start building HTML output
    html_output = "<div style='margin-bottom:10px'>"
    html_output += "<div style='font-weight:bold'>User:</div>"
    html_output += f"<div style='margin-left:20px'>{user_query}</div>"
    html_output += "</div>"

    # Execute the agent response
    response = await agent.on_messages(
        [TextMessage(content=user_query, source="user")],
        cancellation_token=CancellationToken(),
    )

    # Add agent response to HTML
    html_output += "<div style='margin-bottom:20px'>"
    html_output += "<div style='font-weight:bold'>Assistant:</div>"
    html_output += f"<div style='margin-left:20px; white-space:pre-wrap'>{response.chat_message.content}</div>"
    html_output += "</div>"

    # Display formatted HTML
    display(HTML(html_output))

# Run the function
await assistant_run()